# About 

This notebook generates the tables/plots for causative and exploratory attacks against intrusion detection systems. 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt
plt.style.use('ggplot')

## Define Functions for the Notebook 

In [8]:
def plot_exploratory_results(fname:str=None):
    d = np.load(fname, allow_pickle=True)
    d = d['all_pers']

    STATS = ['accs', 'mccs']
    LABELS = ['Accuracy', 'MCC']
    METHOD = ['if', 'svm', 'ee', 'lo']
    x_labels = ['IF', 'SVM', 'EE', 'LO']

    for stat, lab_name in zip(STATS, LABELS): 
        # build gs
        g1, g2, g3, g4, g5 = [], [], [], [], []
        for method in METHOD: 
            g1.append(d[''.join([stat, '_', method, '_baseline'])])
            g2.append(d[''.join([stat, '_', method, '_deepfool'])])
            g3.append(d[''.join([stat, '_', method, '_pgd'])])
            g4.append(d[''.join([stat, '_', method, '_fgsm'])])
            g5.append(d[''.join([stat, '_', method, '_dt'])])
        
    
        x = np.arange(len(x_labels))  # the label locations
        width = 0.15  # the width of the bars
    
        plt.figure()
        fig, ax = plt.subplots()
        rects1 = ax.bar(x - 2*width, g1, width, label='Baseline')
        rects2 = ax.bar(x - 1*width, g2, width, label='Deepfool')
        rects3 = ax.bar(x + 0*width, g3, width, label='PGD')
        rects4 = ax.bar(x + 1*width, g4, width, label='FGSM')
        rects5 = ax.bar(x + 2*width, g5, width, label='DT')

        ax.set_ylabel(lab_name)
        ax.set_xticks(x)
        ax.set_xticklabels(x_labels)
        ax.legend()
        fig.tight_layout()
        plt.savefig(''.join([fname[:-4], '_', stat, '.pdf']) )
        plt.close()
    return None 

In [11]:
def plot_causative_results(fname:str=None): 
    d = np.load(fname, allow_pickle=True)
    d = d['all_perfs']
    
    STATS = ['accs', 'mccs']
    LABELS = ['Accuracy', 'MCC']
    METHOD = ['if', 'svm', 'ee', 'lo']
    x_labels = ['IF', 'SVM', 'EE', 'LO']
    
    for stat, lab_name in zip(STATS, LABELS): 
        # build gs
        g1, g2, g3, g4, g5 = [], [], [], [], []
        for method in METHOD: 
            g1.append(d[''.join([stat, '_', method, '_baseline'])])
            g2.append(d[''.join([stat, '_', method, '_pattern'])])
            g3.append(d[''.join([stat, '_', method, '_single'])])
            g4.append(d[''.join([stat, '_', method, '_svc'])])
        
        x = np.arange(len(x_labels))  # the label locations
        width = 0.15  # the width of the bars
    
        plt.figure()
        fig, ax = plt.subplots()
        rects1 = ax.bar(x - 2*width, g1, width, label='Baseline')
        rects2 = ax.bar(x - 1*width, g2, width, label='BD-Pattern')
        rects3 = ax.bar(x + 0*width, g3, width, label='BD-Single')
        rects4 = ax.bar(x + 1*width, g4, width, label='SVC')
    
        ax.set_ylabel(lab_name)
        ax.set_xticks(x)
        ax.set_xticklabels(x_labels)
        ax.legend()
        fig.tight_layout()
        plt.savefig(''.join([fname[:-4], '_', stat, '.pdf']) )
        plt.close()
    return None 

# Exploratory Attacks 

In [12]:
plot_exploratory_results(fname='outputs/results_ids_attacks_all_nslkdd.npz')
plot_exploratory_results(fname='outputs/results_ids_attacks_only_nslkdd.npz')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# Causative Attacks

In [5]:
plot_causative_results(fname='outputs/results_ids_causative_nslkdd_pp5.npz')
plot_causative_results(fname='outputs/results_ids_causative_nslkdd_pp10.npz')
plot_causative_results(fname='outputs/results_ids_causative_nslkdd_pp15.npz')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>